# 自定义 TVM 自动量化

[](partition) 量化分区（或融合）逻辑很脆弱，可能会引入一些不可预知的问题，为了提供更加健壮的分区逻辑，下面采用模板匹配的策略进行量化分区。

```{admonition} 显式定义融合规则的好处
1. 更加精细控制融合算子的规则
2. 更好的适配诸如 VTA 等后端。
```

以 resnet18 模型为例说明如何自定义量化过程。

In [1]:
import numpy as np
from tvm import relay
import tvm
from tvm_book.tvm_utils.llvm_utils import run_llvm_graph

def load_model(input_shape=[1, 3, 224, 224]):
    """加载前端模型"""
    import torch
    from torchvision.models import resnet18
    from torchvision.models.resnet import ResNet18_Weights
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    data = torch.randn(*input_shape)
    return torch.jit.trace(model.eval(), data)

size = 224, 224
input_shape = (1, 3, *size)
input_name = "data"
traced_model = load_model(input_shape).eval()
# 将前端模型翻译为 relay 模型
origin_mod, origin_params = relay.frontend.from_pytorch(traced_model, [(input_name, input_shape)])

先以 `mod` 子图为例研究定义量化过程：

In [ ]:
from tvm_book.tvm_utils.split_graph import graph_split

split_conf = [{"op_name": "add", "op_index": 0}]
mod = graph_split(origin_mod["main"], split_conf)[0]

其中 `split_conf = [{"op_name": "add", "op_index": 0}]` 取出第一个残差结构所在子图。

此时的 `mod` 存在 `nn.batch_norm` 算子以及常量表达式：

In [3]:
print(mod["main"])

fn (%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */, %aten::_convolution_0.weight: Tensor[(64, 3, 7, 7), float32] /* ty=Tensor[(64, 3, 7, 7), float32] span=aten::_convolution_0.weight:0:0 */, %aten::batch_norm_0.weight: Tensor[(64), float32] /* ty=Tensor[(64), float32] span=aten::batch_norm_0.weight:0:0 */, %aten::batch_norm_0.bias: Tensor[(64), float32] /* ty=Tensor[(64), float32] span=aten::batch_norm_0.bias:0:0 */, %aten::batch_norm_0.running_mean: Tensor[(64), float32] /* ty=Tensor[(64), float32] span=aten::batch_norm_0.running_mean:0:0 */, %aten::batch_norm_0.running_var: Tensor[(64), float32] /* ty=Tensor[(64), float32] span=aten::batch_norm_0.running_var:0:0 */, %aten::_convolution_1.weight: Tensor[(64, 64, 3, 3), float32] /* ty=Tensor[(64, 64, 3, 3), float32] span=aten::_convolution_1.weight:0:0 */, %aten::batch_norm_1.weight: Tensor[(64), float32] /* ty=Tensor[(64), float32] span=aten::batch_norm_1.weight:0

运行如下代码便可将 `nn.batch_norm` 进行融合，同时将其模型参数替换掉常量表达式(还有一些其他操作，此时不展开了)：

In [4]:
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        # 量化前准备
        run_mod = relay.quantize.prerequisite_optimize(mod, origin_params)

In [5]:
print(run_mod["main"])

fn (%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 64, 56, 56), float32] {
  %0 = nn.conv2d(%data, meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), float32] */, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %1 = add(%0, meta[relay.Constant][1] /* ty=Tensor[(64, 1, 1), float32] */) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %2 = nn.relu(%1) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %3 = nn.max_pool2d(%2, pool_size=[3, 3], strides=[2, 2], padding=[1, 1, 1, 1]) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %4 = nn.conv2d(%3, meta[relay.Constant][2] /* ty=Tensor[(64, 64, 3, 3), float32] */, padding=[1, 1, 1, 1], channels=64, kernel_size=[3, 3]) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %5 = add(%4, meta[relay.Constant][3] /* ty=Tensor[(64, 1, 1), float32] */) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %6 = nn.relu(%5)

## 定义融合规则

想要融合 `conv2d+add+relu` 结构，可以定义融合函数：

In [6]:
from tvm.relay.dataflow_pattern import is_op, wildcard

def make_conv_add_relu_pattern():
    """创建如下模式

     conv2d
        |
      (add)
        |
      (relu)
    """
    x = wildcard()
    w = wildcard()
    bias = wildcard()
    r = is_op("nn.conv2d")(x, w)
    r = is_op("add")(r, bias) | r
    # 激活函数
    r = r.optional(lambda x: is_op("nn.relu")(x))
    return r

上述结构模式可以用来匹配 `conv2d`、`conv2d+add`、`conv2d+add+relu`、`conv2d+relu` 四种模式。

执行融合：

In [7]:
compiler_name = "ccompiler"
pattern_table = [
    (f"{compiler_name}.conv_add_relu", make_conv_add_relu_pattern()),
]
merge_passes = tvm.transform.Sequential([
    relay.transform.InferType(),
    relay.transform.MergeComposite(pattern_table),
    # # relay.transform.AnnotateTarget([compiler_name]),
    relay.transform.PartitionGraph(),
])
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        run_mod_f = merge_passes(run_mod)
print(run_mod_f)

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 64, 56, 56), float32] {
  %5 = fn (%FunctionVar_2_0: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %FunctionVar_2_1: Tensor[(64, 3, 7, 7), float32] /* ty=Tensor[(64, 3, 7, 7), float32] */, %FunctionVar_2_2: Tensor[(64, 1, 1), float32] /* ty=Tensor[(64, 1, 1), float32] */, PartitionedFromPattern="nn.conv2d_add_nn.relu_", Composite="ccompiler.conv_add_relu") -> Tensor[(1, 64, 112, 112), float32] {
    %3 = nn.conv2d(%FunctionVar_2_0, %FunctionVar_2_1, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */;
    %4 = add(%3, %FunctionVar_2_2) /* ty=Tensor[(1, 64, 112, 112), float32] */;
    nn.relu(%4) /* ty=Tensor[(1, 64, 112, 112), float32] */
  } /* ty=fn (Tensor[(1, 3, 224, 224), float32], Tensor[(64, 3, 7, 7), float32], Tensor[(64, 1, 1), float32]) -> T

可以看出，上述剩余 `nn.max_pool2d` 和残差 `add` 没有被融合，故此可以添加规则：

In [8]:
def make_max_pool2d_pattern():
    x = wildcard()
    r = is_op("nn.max_pool2d")(x)
    return r

def make_add_pattern():
    return wildcard() + wildcard()


compiler_name = "ccompiler"
# 按照顺序依次执行匹配工作
pattern_table = [
    (f"{compiler_name}.conv_add_relu", make_conv_add_relu_pattern()),
    (f"{compiler_name}.max_pool2d", make_max_pool2d_pattern()),
    (f"{compiler_name}.add", make_add_pattern()),
]
merge_passes = tvm.transform.Sequential([
    relay.transform.InferType(),
    relay.transform.MergeComposite(pattern_table),
    # # relay.transform.AnnotateTarget([compiler_name]),
    relay.transform.PartitionGraph(),
])
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        run_mod_f = merge_passes(run_mod)
print(run_mod_f)

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 64, 56, 56), float32] {
  %5 = fn (%FunctionVar_2_0: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %FunctionVar_2_1: Tensor[(64, 3, 7, 7), float32] /* ty=Tensor[(64, 3, 7, 7), float32] */, %FunctionVar_2_2: Tensor[(64, 1, 1), float32] /* ty=Tensor[(64, 1, 1), float32] */, PartitionedFromPattern="nn.conv2d_add_nn.relu_", Composite="ccompiler.conv_add_relu") -> Tensor[(1, 64, 112, 112), float32] {
    %3 = nn.conv2d(%FunctionVar_2_0, %FunctionVar_2_1, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */;
    %4 = add(%3, %FunctionVar_2_2) /* ty=Tensor[(1, 64, 112, 112), float32] */;
    nn.relu(%4) /* ty=Tensor[(1, 64, 112, 112), float32] */
  } /* ty=fn (Tensor[(1, 3, 224, 224), float32], Tensor[(64, 3, 7, 7), float32], Tensor[(64, 1, 1), float32]) -> T

符合期望结构。

## 为融合函数添加 `QPartitionExpr` 算子

In [9]:
from tvm.relay import Call
from tvm.relay.function import Function, FunctionWithFields
from tvm.relay.quantize._partition import QPartitionExpr

@tvm.relay.transform.function_pass(opt_level=1)
class MergeGraphTransform:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.nodes = []

    def transform_function(self, func, mod, ctx):
        obj = self
        class Replace(tvm.relay.ExprMutator):
            def visit_function(self, fn):
                new_params = [self.visit(x) for x in fn.params]
                new_body = self.visit(fn.body)
                if not isinstance(new_body.op, Function): # 防止循环添加 QPartitionExpr
                    new_body = QPartitionExpr(new_body).realize()
                if new_params == list(fn.params) and new_body == fn.body:
                    new_fn =  fn
                else:
                    new_fn = FunctionWithFields(fn, list(new_params), new_body)
                obj.nodes.append(new_fn)
                return new_fn
        return Replace().visit(func)

In [10]:
transform = MergeGraphTransform()
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        mod_sq = transform(run_mod_f)
mod_sq

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 64, 56, 56), float32] {
  %15 = fn (%FunctionVar_2_0: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %FunctionVar_2_1: Tensor[(64, 3, 7, 7), float32] /* ty=Tensor[(64, 3, 7, 7), float32] */, %FunctionVar_2_2: Tensor[(64, 1, 1), float32] /* ty=Tensor[(64, 1, 1), float32] */, PartitionedFromPattern="nn.conv2d_add_nn.relu_", Composite="ccompiler.conv_add_relu") -> Tensor[(1, 64, 112, 112), float32] {
    %11 = nn.conv2d(%FunctionVar_2_0, %FunctionVar_2_1, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */;
    %12 = add(%11, %FunctionVar_2_2) /* ty=Tensor[(1, 64, 112, 112), float32] */;
    %13 = nn.relu(%12) /* ty=Tensor[(1, 64, 112, 112), float32] */;
    %14 = annotation.cast_hint(%13, dtype="int8") /* ty=Tensor[(1, 64, 112, 112), float32] */;
    ann

## 变换计算图为原语计算图

由于 {class}`tvm.contrib.graph_executor.GraphModule` 不支持对 {class}`tvm.relay.function.Function` 进行推理，需要分解其为原语函数，以支持后续的校准过程：

In [11]:
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        run_mod_sq = relay.transform.DefuseOps()(mod_sq)
run_mod_sq

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 64, 56, 56), float32] {
  %0 = nn.conv2d(%data, meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), float32] */, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %1 = add(%0, meta[relay.Constant][1] /* ty=Tensor[(64, 1, 1), float32] */) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %2 = nn.relu(%1) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %3 = annotation.cast_hint(%2, dtype="int8") /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %4 = annotation.stop_fusion(%3) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %5 = nn.max_pool2d(%4, pool_size=[3, 3], strides=[2, 2], padding=[1, 1, 1, 1]) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %6 = annotation.cast_hint(%5, dtype="int8") /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %7 = annotation.stop_fusion(%6) /* ty=Tensor[(1, 64, 56,

## 注解计算图

In [12]:
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        annotate_mod = relay.quantize.annotate()(run_mod_sq)
print(annotate_mod)

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */, %dom_scale: float32 /* ty=float32 */, %clip_min: float32 /* ty=float32 */, %clip_max: float32 /* ty=float32 */, %dom_scale1: float32 /* ty=float32 */, %clip_min1: float32 /* ty=float32 */, %clip_max1: float32 /* ty=float32 */, %dom_scale2: float32 /* ty=float32 */, %clip_min2: float32 /* ty=float32 */, %clip_max2: float32 /* ty=float32 */, %dom_scale3: float32 /* ty=float32 */, %clip_min3: float32 /* ty=float32 */, %clip_max3: float32 /* ty=float32 */, %dom_scale4: float32 /* ty=float32 */, %clip_min4: float32 /* ty=float32 */, %clip_max4: float32 /* ty=float32 */, %dom_scale5: float32 /* ty=float32 */, %clip_min5: float32 /* ty=float32 */, %clip_max5: float32 /* ty=float32 */, %dom_scale6: float32 /* ty=float32 */, %clip_min6: float32 /* ty=float32 */, %clip_max6: float32 /* ty=float32 */, %dom_scale7: float32 /* ty=float32 */, %clip_min7: float32 /* ty=flo

## 模拟量化

In [13]:
from tvm.relay.quantize import calibrate

# 定义校准数据集
def data_iter(input_name, input_shape, num=1):
    for _ in range(num):
        yield {input_name: np.random.normal(size=input_shape)}

dataset = data_iter(input_name, input_shape)

calibrate_pass = tvm.transform.module_pass(
    calibrate(dataset), opt_level=1, name="QuantizeCalibrate"
)
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        calibrate_mod = calibrate_pass(annotate_mod)
calibrate_mod

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 64, 56, 56), float32] {
  %0 = relay.op.annotation.simulated_quantize(%data, 0.0318054f, -127f, 127f, kind=1) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %1 = relay.op.annotation.simulated_quantize(meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), float32] */, 0.00307715f, -127f, 127f, kind=2) /* ty=Tensor[(64, 3, 7, 7), float32] */;
  %2 = nn.conv2d(%0, %1, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %3 = relay.op.annotation.simulated_quantize(meta[relay.Constant][1] /* ty=Tensor[(64, 1, 1), float32] */, 0.00541632f, -127f, 127f, kind=2) /* ty=Tensor[(64, 1, 1), float32] */;
  %4 = add(%2, %3) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %5 = nn.relu(%4) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %6 = relay.op.annotation.simulated_quantize(%5, 0.0290262f, -127f

## 量化实现

In [14]:
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        run_mod_r = relay.quantize.realize()(calibrate_mod)
run_mod_r

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 64, 56, 56), float32] {
  %0 = multiply(%data, 31.4412f /* ty=float32 */) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %1 = round(%0) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %2 = clip(%1, a_min=-127f, a_max=127f) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %3 = multiply(meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), float32] */, 324.976f /* ty=float32 */) /* ty=Tensor[(64, 3, 7, 7), float32] */;
  %4 = round(%3) /* ty=Tensor[(64, 3, 7, 7), float32] */;
  %5 = clip(%4, a_min=-127f, a_max=127f) /* ty=Tensor[(64, 3, 7, 7), float32] */;
  %6 = cast(%2, dtype="int8") /* ty=Tensor[(1, 3, 224, 224), int8] */;
  %7 = cast(%5, dtype="int8") /* ty=Tensor[(64, 3, 7, 7), int8] */;
  %8 = multiply(meta[relay.Constant][1] /* ty=Tensor[(64, 1, 1), float32] */, 184.627f /* ty=float32 */) /* ty=Tensor[(64, 1, 1), float32] */;
  %9 = round(%8) /* t